<a href="https://colab.research.google.com/github/Bitachi/parking-detector-poc-ipynb/blob/main/parking_detector_poc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics deep_sort_realtime opencv-python-headless --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.3 MB/s eta 0:00:00


In [ ]:
# yolov8_static_car_detection.py

import cv2
import os
import csv
import numpy as np
from ultralytics import YOLO
from collections import defaultdict

# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

# パス設定
PROJECT_DIR = "/content/drive/MyDrive/parking-detector-poc/"
VIDEO_PATH = f"{PROJECT_DIR}/videos/fixedPointCamera.mp4"
MODEL_PATH = f"{PROJECT_DIR}/models/yolov8m.pt"
CSV_PATH = f"/content/stopped_vehicles_yolov8m.csv"
OUTPUT_IMAGE_DIR = "/content/images/yolov8m"

# モデルロード
model = YOLO(MODEL_PATH)

# 出力フォルダ作成
os.makedirs(OUTPUT_IMAGE_DIR, exist_ok=True)

# パラメータ
FRAME_INTERVAL = 1
STOP_THRESHOLD_SECONDS = 10
MAX_SECONDS = 300
IOU_THRESHOLD = 0.5

# 車両追跡用データ
tracking = defaultdict(list)
stopped_vehicles = {}
confirmed_stopped_vehicles = []  # 停止履歴を記録するリスト

# IOU計算
def compute_iou(box1, box2):
    x1, y1, x2, y2 = box1
    x1b, y1b, x2b, y2b = box2

    xi1, yi1 = max(x1, x1b), max(y1, y1b)
    xi2, yi2 = min(x2, x2b), min(y2, y2b)
    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)

    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (x2b - x1b) * (y2b - y1b)
    union_area = box1_area + box2_area - inter_area

    return inter_area / union_area if union_area > 0 else 0

# メイン処理
cap = cv2.VideoCapture(VIDEO_PATH)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_interval = int(fps)
frame_idx = 0
seconds_processed = 0

while cap.isOpened() and seconds_processed < MAX_SECONDS:
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)[0]
    new_tracking = []

    for i, box in enumerate(results.boxes):
        cls_id = int(box.cls[0])
        if cls_id not in [2, 5, 7]:  # 車、バス、トラック
            continue
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        matched = False

        for prev_id, history in tracking.items():
            iou = compute_iou((x1, y1, x2, y2), history[-1][1])
            if iou > IOU_THRESHOLD:
                history.append((seconds_processed, (x1, y1, x2, y2)))

                # 停止判定
                if len(history) >= STOP_THRESHOLD_SECONDS:
                    duration = history[-1][0] - history[0][0]
                    if duration >= STOP_THRESHOLD_SECONDS:
                        stopped_vehicles[prev_id] = {
                            "vehicle_id": prev_id,
                            "start_time": history[0][0],
                            "end_time": history[-1][0],
                            "box": (x1, y1, x2, y2)
                        }

                # 動いたら削除しつつ停止履歴に追加
                if prev_id in stopped_vehicles:
                    stop_box = stopped_vehicles[prev_id]["box"]
                    move_iou = compute_iou((x1, y1, x2, y2), stop_box)
                    if move_iou < IOU_THRESHOLD:
                        # 履歴に追加（重複防止）
                        if stopped_vehicles[prev_id] not in confirmed_stopped_vehicles:
                            confirmed_stopped_vehicles.append(stopped_vehicles[prev_id])
                        del stopped_vehicles[prev_id]

                matched = True
                break

        if not matched:
            tracking[len(tracking)] = [(seconds_processed, (x1, y1, x2, y2))]

        new_tracking.append((x1, y1, x2, y2))

    # 停止車両の描画直前に動き出した車両をリストから除去するフィルターを追加
    stopped_vehicles = {
        vid: vehicle for vid, vehicle in stopped_vehicles.items()
        if vehicle["vehicle_id"] not in [v["vehicle_id"] for v in confirmed_stopped_vehicles]
    }





    # 可視化と保存
    for box in new_tracking:
        cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (255, 0, 0), 2)

    for vehicle in stopped_vehicles.values():
        box = vehicle["box"]
        cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (0, 0, 255), 2)

    frame_time_sec = int(frame_idx / fps)
    img_filename = os.path.join(OUTPUT_IMAGE_DIR, f"frame_{frame_time_sec:04d}.jpg")
    cv2.imwrite(img_filename, frame)

    frame_idx += frame_interval
    seconds_processed += 1

cap.release()

# 動画終了時点でまだ停止中の車両も履歴に追加
for vehicle in stopped_vehicles.values():
    if vehicle not in confirmed_stopped_vehicles:
        confirmed_stopped_vehicles.append(vehicle)

# CSV出力
with open(CSV_PATH, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Vehicle_ID", "Start_Second", "End_Second"])
    for vehicle in confirmed_stopped_vehicles:
        writer.writerow([
            vehicle["vehicle_id"],
            vehicle["start_time"],
            vehicle["end_time"]
        ])

print("処理完了。images/ と stopped_vehicles.csv に出力しました。")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

0: 384x640 4 persons, 2 cars, 1134.8ms
Speed: 2.7ms preprocess, 1134.8ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 3 cars, 804.1ms
Speed: 2.5ms preprocess, 804.1ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 2 cars, 808.7ms
Speed: 2.7ms preprocess, 808.7ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 2 cars, 867.2ms
Speed: 2.7ms preprocess, 867.2ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 3 cars, 812.5ms
Speed: 2.3ms preprocess, 812.5ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 3 cars, 801.2ms
Speed: 2.2ms preprocess, 801.2ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 5 cars, 757.2ms
Speed: 1

In [ ]:
import glob
import os

folder_path = "/content/images/yolov8m"
jpg_files = glob.glob(os.path.join(folder_path, "*.jpg"))

for file in jpg_files:
    os.remove(file)

print(f"✅ {len(jpg_files)} 枚の JPG ファイルを削除しました。")


✅ 300 枚の JPG ファイルを削除しました。
